In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import math
import copy

In [3]:
PATH = "/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /"
dict_web={
    "other":{"list_year":[]},
    "cafef":{"path":PATH+"CafeF/Financial/Financial_F3/Year/BalanceSheet/","money":1000000},
    "VS":{"path":PATH+"Vietstock/Financial/Financial_F3/Year/BalanceSheet/","money":1000},
    "68":{"path":PATH+"Cổ phiếu 68/Financial/Financial_F3/Year/BalanceSheet/","money":1}
}
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p_phong = f'{p}Phong /'
data_field = pd.read_excel(f'{p_phong}Compare_Feature.xlsx', sheet_name = 'Balance Sheet_ Vang + Phong ')
data_field = data_field[["Feature"]][:59].reset_index(drop=True)

In [4]:
def get_field_basic():
  return copy.deepcopy(data_field)
  
def getDataCompany(symbol):
  min = 1
  dict_web["other"]["list_year"] = []
  data_field = get_field_basic()
  for key in dict_web.keys():
      if key == "other":
        continue
      df = pd.read_csv("{}{}.csv".format(dict_web[key]["path"],symbol))

      if len(df.columns)> min:
        min = len(df)
        dict_web["other"]["list_year"] = df.columns[1:]
      for column in dict_web["other"]["list_year"]:
          if not(column in df.columns):
              df[column]=[np.nan for i in df["Feature"]]
      dict_web[key]["company"] = pd.merge(data_field,df,on="Feature",how="left").fillna(0)
getDataCompany("HAP")

In [5]:
def check_dau(a):
  if a >=0:
    return 1
  else:
    return -1

def compare_2_field(a,b,s_a,s_b):
    dau_a = check_dau(a)
    dau_b = check_dau(b)
    a = abs(a)
    b = abs(b)
    if abs(s_a - s_b) == 999999:
      if s_b == 1:
        x = round(a/1000+0.0000001)/1000+0.0000001
        y = b
      else:
        x = a
        y = round(b/1000+0.0000001)/1000+0.0000001

    elif abs(s_a - s_b) == 999000:
      if s_b == 1000:
        x = round(a/1000+0.0000001)/1000+0.0000001
        y = b/1000+0.0000001
      else:
        x = a/1000+0.0000001
        y = round(b/1000+0.0000001)/1000+0.0000001
    
    elif abs(s_a - s_b) == 999:
      x = a/s_a+0.0000001
      y = b/s_b+0.0000001
    else:
      print("dell hieu kieu gi")
    x = dau_a*x
    y = dau_b*y
    # print(a,b,x,y,round(x),round(y))

    if round(x)-round(y) == 0:
      if a*b == 0 and a-b !=0:
        return "N"
      return "1"
    elif a*b ==0:
      return "N"
    else:
      if a*b != 0:
        return "0"
      else:
        return "N"


In [6]:
def get_field(key_1,key_2):
  df = pd.merge(dict_web[key_1]["company"],dict_web[key_2]["company"],on=["Feature"],how="inner").fillna(0)
  list_year = dict_web["other"]["list_year"]
  s_a,s_b = dict_web[key_1]["money"],dict_web[key_2]["money"]
  for year in list_year:
    df[year] = df.apply(lambda row: compare_2_field(row[f"{year}_x"],row[f"{year}_y"],s_a,s_b),axis=1)
    df = df.drop(columns=[f"{year}_x",f"{year}_y"])
  return df

In [7]:
def render(year):
  return [year+"_x",year+"_y",year]

def ghep(a,b,c):
  return "{}_{}_{}".format(a,b,c)

In [8]:
def compare(symbol):
  getDataCompany(symbol)
  a = get_field("cafef","68")
  b = get_field("VS","68")
  c = get_field("cafef","VS")
  f = a.merge(b,on="Feature",how="outer")
  data_ = f.merge(c,on="Feature",how="outer")
  list_year = dict_web["other"]["list_year"]
  for i in list_year:
    y = render(i)
    data_[i+"_"] = data_.apply(lambda row: ghep(row[y[0]],row[y[1]],row[y[2]]),axis=1)
    data_ = data_.drop(columns=y)
  return data_
# compare('CID')

In [9]:
p_compare = f'{p_phong}/compare/'
path_lake = '/content/drive/MyDrive/Data Lake/'
# all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_First (1_4).xlsx')['Symbol'])
# all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase2.xlsx')['Symbol'])
# all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase3.xlsx')['Symbol'])
all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase4.xlsx')['Symbol'])
for i in range(len(all_com)):
    symbol = all_com[i]
    print(i, symbol, end= ' ')
    compare(symbol).to_csv(f'{p_compare}{symbol}.csv', index = False)
    print('Done')

0 SJ1 Done
1 SJC Done
2 SJD Done
3 SJE Done
4 SJF Done
5 SJG Done
6 SJM Done
7 SJS Done
8 SKG Done
9 SKH Done
10 SKN Done
11 SKS Done
12 SKV Done
13 SLC Done
14 SLD Done
15 SLS Done
16 SMA Done
17 SMB Done
18 SMC Done
19 SMN Done
20 SMT Done
21 SNC Done
22 SNG Done
23 SNP Done
24 SNZ Done
25 SON Done
26 SOV Done
27 SP2 Done
28 SPA Done
29 SPB Done
30 SPC Done
31 SPD Done
32 SPH Done
33 SPI Done
34 SPM Done
35 SPP Done
36 SPS Done
37 SPT Done
38 SPV Done
39 SQC Done
40 SRA Done
41 SRB Done
42 SRC Done
43 SRF Done
44 SRT Done
45 SSC Done
46 SSE Done
47 SSF Done
48 SSG Done
49 SSH Done
50 SSM Done
51 SSN Done
52 SSS Done
53 SST Done
54 SSU Done
55 ST8 Done
56 STC Done
57 STD Done
58 STG Done
59 STH Done
60 STJ Done
61 STK Done
62 STL Done
63 STN Done
64 STP Done
65 STS Done
66 STT Done
67 STU Done
68 STV Done
69 STW Done
70 SUM Done
71 SVC Done
72 SVD Done
73 SVG Done
74 SVH Done
75 SVI Done
76 SVJ Done
77 SVL Done
78 SVN Done
79 SVT Done
80 SWC Done
81 SZA Done
82 SZB Done
83 SZC Done
84

#checkHieu


In [ ]:
path_cf = '/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /CafeF/Financial/Financial_F3/Year/IncomeStatement/'
path_68 = '/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /Cổ phiếu 68/Financial/Financial_F3/Year/IncomeStatement/'
path_vs = '/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /Vietstock/Financial/Financial_F3/Year/IncomeStatement/'
data_field = pd.read_csv('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Cổ phiếu 68/HIEUCODE/full_feature_income.csv')

In [ ]:
PATH = "/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /"
dict_web={
    "other":{"list_year":[]},
    "cafef":{"path":'/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /CafeF/Financial/Financial_F3/Year/IncomeStatement/',"money":1000000},
    "VS":{"path":'/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /Vietstock/Financial/Financial_F3/Year/IncomeStatement/',"money":1000},
    "68":{"path":'/content/drive/MyDrive/Data Lake/Raw_VIS/Day 0 /Cổ phiếu 68/Financial/Financial_F3/Year/IncomeStatement/',"money":1}
}

In [ ]:
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p_phong = f'{p}Phong /'
p_compare = f'{p_phong}/compareHieu/'
all_com = list(pd.read_excel('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /List_Com_First (1_4).xlsx')['Symbol'])
for i in range(len(all_com)):
    symbol = all_com[i]
    print(i, symbol, end= ' ')
    compare(symbol).to_csv(f'{p_compare}{symbol}.csv', index = False)
    print('Done')